### 📥 Setups

Installation and Import of required packages

In [70]:
# !pip install -r requirements.txt

In [71]:
import yfinance as yf
import pandas as pd
import numpy as np

Initialization of Analysis Parameters

In [72]:
TICKERS: list[str] = ["AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "JNJ", "JPM", "XOM", "PG", "WMT", "AMT"]
START_DATE: str = "2022-01-01"
END_DATE: str = "2025-07-18"

### 🏗️ Data Acquisition

Function for Fetching Daily Price Data

In [ ]:
def get_stock_data(tickers: list[str], start: str, end: str):
    """
    Fetches financial data from Yahoo Finance API for given tickers and date range.
    
    Parameters:
        tickers (list[str]): U.S. equity ticker symbols
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: DataFrame with date-indexed adjusted closing price data
    """
    
    data = {}

    for ticker in tickers:
        try:
            # Fetch data from Yahoo Finance and select relevant columns
            df = yf.download(ticker, start=start, end=end, auto_adjust=True)[['Close', 'Volume']]
            df.columns = ['Close', 'Volume']
            
            if not df.empty:
                data[ticker] = df
            else:
                print(f"No data found for {ticker} in the specified date range.")
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data